# Using_an_SBML_model (python3)

This is a copy of Using_an_SBML_model (Python2) but built with a Python3 kernel to make sure everything works.

# Using an SBML model

## Getting started

### Installing libraries

Before you start, you will need to install a couple of libraries:
   
The [ModelSeedDatabase](https://github.com/ModelSEED/ModelSEEDDatabase) has all the biochemistry we'll need. You can install that with `git clone`.
   
The [PyFBA](http://linsalrob.github.io/PyFBA) library has detailed [installation instructions](http://linsalrob.github.io/PyFBA/installation.html). Don't be scared, its mostly just `pip install`.

(Optional) Also, get the [SEED Servers](https://github.com/linsalrob/SEED_Servers_Python) as you can get a lot of information from them. You can install the git python repo from github.  Make sure that the SEED_Servers_Python is in your PYTHONPATH.

We start with importing some modules that we are going to use. 

We import *sys* so that we can use standard out and standard error if we have some error messages.<br>
We import *copy* so that we can make a deep copy of data structures for later comparisons.<br>
Then we import the *PyFBA* module to get started.

In [1]:
import sys
import os
import copy
import PyFBA
import pickle

## Running an SBML model

If you have run your genome through RAST, you can download the [SBML](http://www.sbml.org/) model and use that directly.

We have provided an [SBML model of *Citrobacter sedlakii*](https://raw.githubusercontent.com/linsalrob/PyFBA/master/example_data/Citrobacter/Citrobacter_sedlakii.sbml) that you can download and use. You can right-ctrl click on this link and save the SBML file in the same location you are running this iPython notebook.

We use this SBML model to demonstrate the key points of the FBA approach: defining the reactions, including the boundary, or drainflux, reactions; the compounds, including the drain compounds; the media; and the reaction bounds. 

We'll take it step by step!

We start by parsing the model:

In [2]:
sbml = PyFBA.parse.parse_sbml_file("../example_data/Citrobacter/Citrobacter_sedlakii.sbml")

We are logging to /home/redwards/GitHubsLinux/PyFBA/iPythonNotebooks/PyFBA.2021-06-10T21:49:18.530785.log
Found rxn05368: {'id': 'rxn05368', 'model_seed_id': 'rxn05368', 'readable_name': '8_methyl_3_oxo_decanoyl_ACP_NADP_oxidoreductase_c0', 'description': None, 'equation': ' (1) cpd00067: H (location: c) +  (1) cpd11504: 8_methyl_3_oxo_decanoyl_ACP (location: c) +  (1) cpd00005: NADPH (location: c) =  (1) cpd00006: NADP (location: c) +  (1) cpd11505: 8_methyl_3_hydroxy_decanoyl_ACP (location: c)', 'direction': '=', 'left_compounds': {<PyFBA.metabolism.compound.CompoundWithLocation object at 0x7f3a764d2a90>, <PyFBA.metabolism.compound.CompoundWithLocation object at 0x7f3a764d2bb0>, <PyFBA.metabolism.compound.CompoundWithLocation object at 0x7f3a764d2af0>}, 'left_abundance': {<PyFBA.metabolism.compound.CompoundWithLocation object at 0x7f3a764d2a90>: 1.0, <PyFBA.metabolism.compound.CompoundWithLocation object at 0x7f3a764d2af0>: 1.0, <PyFBA.metabolism.compound.CompoundWithLocation object 

In [3]:
import pprint
pp = pprint.PrettyPrinter(indent=4)
pp.pprint(sbml.reactions['rxn05368'].__dict__)

{   'biomass_direction': False,
    'deltaG': 0,
    'deltaG_error': 0,
    'description': None,
    'direction': '=',
    'ec_numbers': [],
    'enzymes': set(),
    'equation': ' (1) cpd00067: H (location: c) +  (1) cpd11504: '
                '8_methyl_3_oxo_decanoyl_ACP (location: c) +  (1) cpd00005: '
                'NADPH (location: c) =  (1) cpd00006: NADP (location: c) +  '
                '(1) cpd11505: 8_methyl_3_hydroxy_decanoyl_ACP (location: c)',
    'gapfill_method': '',
    'id': 'rxn05368',
    'inp': False,
    'is_biomass_reaction': False,
    'is_gapfilled': False,
    'is_transport': False,
    'is_uptake_secretion': False,
    'left_abundance': {   <PyFBA.metabolism.compound.CompoundWithLocation object at 0x7f3a764d2bb0>: 1.0,
                          <PyFBA.metabolism.compound.CompoundWithLocation object at 0x7f3a764d2a90>: 1.0,
                          <PyFBA.metabolism.compound.CompoundWithLocation object at 0x7f3a764d2af0>: 1.0},
    'left_compounds': {   <P

In [4]:
for c in sbml.reactions['rxn05368'].left_compounds:
    print(c)

cpd00067: H (location: c)
cpd00005: NADPH (location: c)
cpd11504: 8_methyl_3_oxo_decanoyl_ACP (location: c)


### Find all the reactions and identify those that are boundary reactions

We need a set of reactions to run in the model. In this case, we are going to run all the reactions in our SBML file. However, you can change this set if you want to knock out reactions, add reactions, or generally modify the model. We store those in the `reactions_to_run` set.

The boundary reactions refer to compounds that are secreted but then need to be removed from the `reactions_to_run` set. We usually include a consumption of those compounds that is open ended, as if they are draining away. We store those reactions in the `uptake_secretion_reactions` dictionary.


In [5]:
# Get a dict of reactions.
# The key is the reaction ID, and the value is a metabolism.reaction.Reaction object
reactions = sbml.reactions
reactions_to_run = set()
uptake_secretion_reactions = {}
biomass_equation = None
for r in reactions:
    if 'biomass_equation' == r:
        biomass_equation = reactions[r]
        print(f"Our biomass equation is {biomass_equation.readable_name}")
        continue
    is_boundary = False
    for c in reactions[r].all_compounds():
        if c.uptake_secretion:
            is_boundary = True
            break
    if is_boundary:
        reactions[r].is_uptake_secretion = True
        uptake_secretion_reactions[r] = reactions[r]
    else:
        reactions_to_run.add(r)


Our biomass equation is Citrobacter_sedlakii_119_auto_biomass


At this point, we can take a look at how many reactions are in the model, not counting the biomass reaction:

In [6]:
print(f"The biomass equation is {biomass_equation}")
print("There are {} reactions in the model".format(len(reactions)))
print("There are {}".format(len(uptake_secretion_reactions)),
      "uptake/secretion reactions in the model")
print("There are {}".format(len(reactions_to_run)),
      "reactions to be run in the model")

The biomass equation is biomass_equation: Citrobacter_sedlakii_119_auto_biomass
There are 1574 reactions in the model
There are 0 uptake/secretion reactions in the model
There are 1573 reactions to be run in the model


In [7]:
import pickle
with open('rgood.txt', 'w') as out:
    for r in reactions:
        out.write(f"{r}\n")

### Find all the compounds in the model, and filter out those that are secreted

We need to filter out uptake and secretion compounds from our list of all compounds before we can make a stoichiometric matrix.

In [8]:
# Get a dict of compounds. 
# The key is the string representation of the compound and
# the value is a metabolite.compound.Compound object
# Get a dict of compounds. 
# The key is the string representation of the compound and
# the value is a metabolite.compound.Compound object
all_compounds = sbml.compounds
# Filter for compounds that are boundary compounds
filtered_compounds = set()
for c in all_compounds:
    if not c.uptake_secretion:
        filtered_compounds.add(c)

Again, we can see how many compounds there are in the model.

In [9]:
print("There are {} total compounds in the model".format(len(all_compounds)))
print("There are {}".format(len(filtered_compounds)),
      "compounds that are not involved in uptake and secretion")

There are 1475 total compounds in the model
There are 1301 compounds that are not involved in uptake and secretion


And now we have the size of our stoichiometric matrix! Notice that the stoichiometric matrix is composed of the reactions that we are going to run and the compounds that are in those reactions (but not the uptake/secretion reactions and compounds).

In [10]:
print(f"The stoichiometric matrix will be {len(reactions_to_run):,} reactions by {len(filtered_compounds):,} compounds")

The stoichiometric matrix will be 1,573 reactions by 1,301 compounds


### Read the media file, and correct the media names

In our [media](https://github.com/linsalrob/PyFBA/tree/master/media) directory, we have a lot of different media formulations, most of which we use with the Genotype-Phenotype project. For this example, we are going to use Lysogeny Broth (LB). There are many different formulations of LB, but we have included the recipe created by the folks at Argonne so that it is comparable with their analysis. You can download [ArgonneLB.txt](https://raw.githubusercontent.com/linsalrob/PyFBA/master/media/ArgonneLB.txt) and put it in the same directory as this iPython notebook to run it.

Once we have read the file we need to correct the names in the compounds. Sometimes when compound names are exported to the SBML file they are modified slightly. This just corrects those names.

In [11]:
# Read the media file
media = PyFBA.parse.read_media_file("/home/redwards/.local/lib/python3.9/site-packages/PyFBA-2.1-py3.9.egg/PyFBA/Biochemistry/media/ArgonneLB.txt")
# Correct the names
media = sbml.correct_media(media)
print(f"The media has {len(media)} compounds")

The media has 65 compounds


Checking media compounds: Our compounds do not include  Ni2+
Checking media compounds: Our compounds do not include  Vitamin B12
Checking media compounds: Our compounds do not include  chromate
Checking media compounds: Our compounds do not include  L-Cystine
Checking media compounds: Our compounds do not include  Molybdate
Checking media compounds: Our compounds do not include  Thiamine phosphate
Please note: The warnings about media not being found in compounds are not fatal.
It just means that we did not find that compound anywhere in the reactions, and so it is unlikely to be
needed or used. We typically see a few of these in rich media.


In [12]:
print(f"There are {len(filtered_compounds):,} filtered compounds, {len(all_compounds):,} all compounds, and {len(reactions):,} reactions")

There are 1,301 filtered compounds, 1,475 all compounds, and 1,574 reactions


### Set the reaction bounds for uptake/secretion compounds

The uptake and secretion compounds typically have reaction bounds that allow them to be consumed (i.e. diffuse away from the cell) but not produced. However, our media components can also increase in concentration (i.e. diffuse to the cell) and thus the bounds are set higher. Whenever you change the growth media, you also need to adjust the reaction bounds to ensure that the media can be consumed!


In [13]:
# Adjust the lower bounds of uptake secretion reactions
# for things that are not in the media
for u in uptake_secretion_reactions:
    is_media_component = False
    for c in uptake_secretion_reactions[u].all_compounds():
        if c in media:
            is_media_component = True
    if not is_media_component:
        reactions[u].lower_bound = 0.0
        uptake_secretion_reactions[u].lower_bound = 0.0

### Run the FBA

Now that we have constructed our model, we can run the FBA!

In [14]:
status, value, growth = PyFBA.fba.run_fba(filtered_compounds, reactions,
                                          reactions_to_run, media, biomass_equation,
                                          uptake_secretion_reactions, verbose=True)
print("The FBA completed with a flux value of {} --> growth: {}".format(value, growth))


create_stoichiometric_matrix is adding compound cpd00393: Folate (location: e) from media to compounds
create_stoichiometric_matrix is adding compound cpd00091: UMP (location: e) from media to compounds
create_stoichiometric_matrix is adding compound cpd00239: H2S (location: e) from media to compounds
create_stoichiometric_matrix is adding compound Media064: Ni2+ (location: e) from media to compounds
create_stoichiometric_matrix is adding compound cpd00215: Pyridoxal (location: e) from media to compounds
create_stoichiometric_matrix is adding compound Media001: Vitamin B12 (location: e) from media to compounds
create_stoichiometric_matrix is adding compound cpd00046: CMP (location: e) from media to compounds
create_stoichiometric_matrix is adding compound Media047: chromate (location: e) from media to compounds
create_stoichiometric_matrix is adding compound cpd00218: Niacin (location: e) from media to compounds
create_stoichiometric_matrix is adding compound cpd00018: AMP (location: e

The FBA completed with a flux value of 281.8417574366177 --> growth: True


# Export the components of the model

I am trying to compare this model to one built just from reactions, and so to see what's working, I'm going to export all the components and then import them as need.

Went down a big rabbit hole on this one, because `reactions` and `compounds` are both recursive (`reactions` contain `compounds` and `compounds` are involved in `reactions`). The following code block fails for some compounds with an `AttributeError`:

```
for c in filtered_compounds:
    t = copy.deepcopy(c)
```

The same error occurs during pickling.

We initially could not solve this by a shallow copy of the data, mainly because `reaction.left_compounds` and `reaction.left_compound_abundance` end up not being the same thing, but I solved that by implementing `__setstate__` and `__getstate__` and now we can pickle and unpickle the data and the FBA still solves.

In [15]:
ec={}
ma = []
for c in filtered_compounds:
    try:
        pickle.dump(c, open('testcpd.pickle', 'wb'), protocol=pickle.HIGHEST_PROTOCOL)
        l = pickle.load(open('testcpd.pickle', 'rb'))
        if c.name == 'Myristic_acid':
            d = c.__dict__.copy()
            d['pickle'] = 'passed'
            ma.append(d)
    except AttributeError:
        print(f"Error with {c}: |{c.name}|")
        if c.name == 'Myristic_acid':
            d = c.__dict__.copy()
            d['pickle'] = 'failed'
            ma.append(d)

print(ma)
    

[{'id': 'cpd03847', 'name': 'Myristic_acid', 'reactions': set(), 'model_seed_id': 'cpd03847', 'alternate_seed_ids': set(), 'abbreviation': 'cpd03847_c0', 'aliases': None, 'formula': None, 'mw': 0, 'common': False, 'charge': '0', 'is_cofactor': False, 'linked_compound': False, 'pka': 0, 'pkb': 0, 'is_obsolete': False, 'abstract_compound': False, 'uptake_secretion': False, 'is_core': False, 'inchikey': 0, 'location': 'c', 'pickle': 'passed'}, {'id': 'cpd03847', 'name': 'Myristic_acid', 'reactions': set(), 'model_seed_id': 'cpd03847', 'alternate_seed_ids': set(), 'abbreviation': 'cpd03847_e0', 'aliases': None, 'formula': None, 'mw': 0, 'common': False, 'charge': '0', 'is_cofactor': False, 'linked_compound': False, 'pka': 0, 'pkb': 0, 'is_obsolete': False, 'abstract_compound': False, 'uptake_secretion': False, 'is_core': False, 'inchikey': 0, 'location': 'e', 'pickle': 'passed'}]


In [16]:
pickle.dump(filtered_compounds, open('compounds.pickle', 'wb'), protocol=pickle.HIGHEST_PROTOCOL)
pickle.dump(reactions, open('reactions.pickle', 'wb'))
pickle.dump(reactions_to_run, open('reactions_to_run.pickle', 'wb'))
pickle.dump(media, open('media.pickle', 'wb'))
pickle.dump(biomass_equation, open('sbml_biomass.pickle', 'wb'))
pickle.dump(uptake_secretion_reactions, open('uptake_secretion_reactions.pickle', 'wb'))

In [17]:
sbml_filtered_compounds = pickle.load(open('compounds.pickle', 'rb'))
sbml_reactions = pickle.load(open('reactions.pickle', 'rb'))
sbml_reactions_to_run = pickle.load(open('reactions_to_run.pickle', 'rb'))
sbml_media = pickle.load(open('media.pickle', 'rb'))
sbml_biomass_equation = pickle.load(open('sbml_biomass.pickle', 'rb'))
sbml_uptake_secretion_reactions = pickle.load(open('uptake_secretion_reactions.pickle', 'rb'))

In [18]:
status, value, growth = PyFBA.fba.run_fba(sbml_filtered_compounds, sbml_reactions,
                                          sbml_reactions_to_run, sbml_media, sbml_biomass_equation,
                                          sbml_uptake_secretion_reactions, verbose=True)
print("The FBA completed with a flux value of {} --> growth: {}".format(value, growth))


create_stoichiometric_matrix found 190 uptake and secretion reactions
In the model there are : 1493 compounds and 1764 reactions
We are loading 1493 rows and 1764 columns


The FBA completed with a flux value of 281.8417574366177 --> growth: True


Length of the media: 65
Number of reactions to run: 1573
Number of compounds in SM: 1493
Number of reactions in SM: 1764
Revised number of total reactions: 1764
Number of total compounds: 1319
SMat dimensions: 1493 x 1764


In [19]:
for r in sbml_reactions:
    for c in sbml_reactions[r].all_compounds():
        if '_' in c.name:
            print(c.name)

L_Alanine
met_L_ala_L
L_Methionine
L_Threonine
L_Alanine
ala_L_Thr_L
gly_asp_L
L_Aspartate
Gly_Cys
L_Cysteine
L_Alanine
ala_L_glu_L
L_Glutamate
gly_asn_L
L_Asparagine
Gly_Leu
L_Leucine
L_Alanine
L_alanylglycine
L_Alanine
ala_L_asp_L
L_Aspartate
Cys_Gly
L_Cysteine
Gly_Gln
L_Glutamine
Gly_Phe
L_Phenylalanine
L_Histidine
L_Alanine
Ala_His
gly_glu_L
L_Glutamate
Ala_Gln
L_Alanine
L_Glutamine
L_Tyrosine
Gly_Tyr
gly_pro_L
L_Proline
Ala_Leu
L_Alanine
L_Leucine
Gly_Met
L_Methionine
Glucose_1_phosphate
Crotonyl_CoA
Butyryl_CoA
Methacrylyl_CoA
Isobutyryl_CoA
Tiglyl_CoA
2_Methylbutyryl_CoA
D_Glucosamine1_phosphate
N_Acetyl_D_glucosamine1_phosphate
Acetyl_CoA
N_Acetyl_D_glucosamine1_phosphate
UDP_N_acetylglucosamine
L_Argininosuccinate
L_Arginine
4_Carboxymuconolactone
3_oxoadipate_enol_lactone
2_Demethylmenaquinone_8
2_Demethylmenaquinol_8
Ubiquinol_8
Ubiquinone_8
Menaquinol_8
Menaquinone_8
1_anteisoheptadecanoyl_sn_glycerol_3_phosphate
1_2_dianteisoheptadecanoyl_sn_glycerol_3_phosphate
1_isohexad

1_2_didodecanoyl_sn_glycerol_3_phosphate
CDP_1_2_didodecanoylglycerol
1_2_dianteisopentadecanoyl_sn_glycerol_3_phosphate
CDP_1_2_dianteisopentadecanoylglycerol
CDP_1_2_dihexadec_9_enoylglycerol
1_2_dihexadec_9_enoyl_sn_glycerol_3_phosphate
CDP_1_2_diisoheptadecanoylglycerol
1_2_diisoheptadecanoyl_sn_glycerol_3_phosphate
CDP_1_2_diisoheptadecanoylglycerol
Glycerol_3_phosphate
CDP_1_2_dihexadecanoylglycerol
Phosphatidylglycerophosphate_dihexadecanoyl
Glycerol_3_phosphate
CDP_1_2_dianteisopentadecanoylglycerol
Glycerol_3_phosphate
Phosphatidylglycerophosphate_dioctadec_11_enoyl
CDP_1_2_dioctadec_11_enoylglycerol
Glycerol_3_phosphate
CDP_1_2_ditetradec_7_enoylglycerol
Phosphatidylglycerophosphate_ditetradec_7_enoyl
Glycerol_3_phosphate
CDP_1_2_dioctadecanoylglycerol
Phosphatidylglycerophosphate_dioctadecanoyl
Glycerol_3_phosphate
CDP_1_2_diisohexadecanoylglycerol
Glycerol_3_phosphate
Phosphatidylglycerophosphate_didodecanoyl
CDP_1_2_didodecanoylglycerol
Glycerol_3_phosphate
CDP_1_2_diisote

Lauroyl_KDO2_lipid_IVA
kdo2_lipid_iva
Dodecanoyl_ACP
Guanosine_5_triphosphate_3_diphosphate
L_Argininosuccinate
L_Aspartate
L_Methionine
5_Methyltetrahydrofolate
D_Xylulose5_phosphate
D_Ribulose5_phosphate
4__cytidine5_diphospho_2_C_methyl_D_erythritol
2_phospho_4__cytidine5_diphospho_2_C_methyl_D_erythritol
Protein_dithiol
Protein_disulfide
L_Malate
L_Malate
L_Malate
L_Malate
2_Oxoglutarate
2_Oxoglutarate
5_Dehydroquinate
3_Dehydroshikimate
6_hydroxymethyl_dihydropterin
7_8_Dihydropterin_pyrophosphate
Glycerone_phosphate
Glycerol_3_phosphate
Glycerone_phosphate
Menaquinone_8
Menaquinol_8
Glycerol_3_phosphate
Glycerone_phosphate
Ubiquinone_8
Ubiquinol_8
Glycerol_3_phosphate
Glycerone_phosphate
2_Demethylmenaquinone_8
2_Demethylmenaquinol_8
Glycerol_3_phosphate
3_Isopropylmalate
2_isopropyl_3_oxosuccinate
LL_2_6_Diaminopimelate
N_Succinyl_L_2_6_diaminopimelate
D_Glucose
beta_D_Glucose
cis_Aconitate
cis_Aconitate
D_Fructuronate
D_Mannonate
beta_Alanine
Glycerol_3_phosphate
Glycerol_3_pho

3_phosphoshikimate
L_Aspartate
tRNA_Glu
L_Glutamate1_semialdehyde
L_Glutamyl_tRNA_Glu
beta_D_Glucose_1_phosphate
beta_D_Glucose_6_phosphate
L_Methionine
S_Adenosyl_L_methionine
L_Glutamate
L_Glutamine
tRNA_Glu
L_Glutamate
L_Glutamyl_tRNA_Glu
2_Phosphoglycolate
L_Malate
N_5_phosphoribosyl_anthranilate
1_2_carboxyphenylamino_1_deoxyribulose_5_phosphate
S_Adenosyl_L_methionine
S_Adenosylmethioninamine
L_Threonine
L_Threonine
L_Aspartate4_semialdehyde
S_Adenosyl_homocysteine
S_Adenosyl_L_methionine
trans_Aconitate
E_3_Methoxycarbonylpent_2_enedioate
5_Phosphoribosyl_4_carboxy_5_aminoimidazole
Biotinyl_5_AMP
UDP_N_acetylmuramoyl_L_alanyl_D_glutamyl_6_carboxy_L_lysyl_D_alanyl__D_alanine
Undecaprenyl_diphospho_N_acetylmuramoyl_L_alanyl_D_glutamyl_meso_2_6_diaminopimeloyl_D_alanyl_D_alanine
MurAcoyl_L_Ala_D_gamma_Glu_L_Lys_D_Ala_D_Ala_diphospho__undecaprenol
UDPMurAcoyl_L_Ala_D_gamma_Glu_L_Lys_D_Ala_D_Ala
2_Acetamido_5_oxopentanoate
n_acetylglutamyl_phosphate
ADP_L_glycero_D_manno_heptose
hept

L_Malate
L_Malate
2_keto_3_deoxygluconate
2_keto_3_deoxygluconate
L_Cysteate
L_Cysteate
D_Galacturonate
D_Galacturonate
Nicotinamide_ribonucleotide
Nicotinamide_ribonucleotide
L_Threonine
L_Threonine
D_Ribose
D_Ribose
L_Isoleucine
L_Isoleucine
gly_asn_L
gly_asn_L
R_3_Hydroxybutanoate
R_3_Hydroxybutanoate
Vitamin_B12r
Vitamin_B12r
L_methionine_R_oxide
L_methionine_R_oxide
L_Tyrosine
L_Tyrosine
Gly_Tyr
Gly_Tyr
Ala_Leu
Ala_Leu
D_Fructose
D_Fructose
met_L_ala_L
met_L_ala_L
L_Asparagine
L_Asparagine
Ala_His
Ala_His
L_Lysine
L_Lysine
N_Acetyl_D_glucosamine
N_Acetyl_D_glucosamine
D_Mucic_acid
D_Mucic_acid
L_Histidine
L_Histidine
D_Glucose
D_Glucose
L_Serine
L_Serine
L_Phenylalanine
L_Phenylalanine
L_Aspartate
L_Aspartate
gly_glu_L
gly_glu_L
L_Leucine
L_Leucine
L_Proline
L_Proline
D_Methionine
D_Methionine
D_Arabinose
D_Arabinose
Vitamin_B12
Vitamin_B12
Myristic_acid
Myristic_acid
gly_pro_L
gly_pro_L
L_Valine
L_Valine
D_Galactonate
D_Galactonate
D_Alanine
D_Alanine
L_Glutamate
L_Glutamate
Gly_

In [20]:
for r in sbml_reactions:
    print(sbml_reactions[r].equation)

 (1) cpd00067: H (location: c) +  (1) cpd00001: H2O (location: c) +  (1) cpd11590: met_L_ala_L (location: c) =  (1) cpd00060: L_Methionine (location: c) +  (1) cpd00035: L_Alanine (location: c)
 (1) cpd00001: H2O (location: c) +  (1) cpd11582: ala_L_Thr_L (location: c) =  (1) cpd00161: L_Threonine (location: c) +  (1) cpd00035: L_Alanine (location: c)
 (1) cpd11589: gly_asp_L (location: c) +  (1) cpd00001: H2O (location: c) =  (1) cpd00041: L_Aspartate (location: c) +  (1) cpd00033: Glycine (location: c)
 (1) cpd15603: Gly_Cys (location: c) +  (1) cpd00001: H2O (location: c) =  (1) cpd00084: L_Cysteine (location: c) +  (1) cpd00033: Glycine (location: c)
 (1) cpd00001: H2O (location: c) +  (1) cpd11586: ala_L_glu_L (location: c) =  (1) cpd00023: L_Glutamate (location: c) +  (1) cpd00035: L_Alanine (location: c)
 (1) cpd11581: gly_asn_L (location: c) +  (1) cpd00001: H2O (location: c) =  (1) cpd00132: L_Asparagine (location: c) +  (1) cpd00033: Glycine (location: c)
 (1) cpd15604: Gly_L

 (1) cpd15754: Isohexadecanoyllipoteichoic_acid_n24__linked__unsubstituted (location: c) +  (24) cpd00037: UDP_N_acetylglucosamine (location: c) =  (24) cpd00014: UDP (location: c) +  (1) cpd15772: Isohexadecanoyllipoteichoic_acid_n24__linked__N_acetyl_D_glucosamine (location: c)
 (1) cpd15748: Stearoyllipoteichoic_acid_n24__linked__unsubstituted (location: c) +  (24) cpd00037: UDP_N_acetylglucosamine (location: c) =  (24) cpd00014: UDP (location: c) +  (1) cpd15766: Stearoyllipoteichoic_acid_n24__linked__N_acetyl_D_glucosamine (location: c)
 (1) cpd15746: Palmitoyllipoteichoic_acid_n24__linked__unsubstituted (location: c) +  (24) cpd00037: UDP_N_acetylglucosamine (location: c) =  (24) cpd00014: UDP (location: c) +  (1) cpd15764: Palmitoyllipoteichoic_acid_n24__linked__N_acetyl_D_glucosamine (location: c)
 (1) cpd15751: Isotetradecanoyllipoteichoic_acid_n24__linked__unsubstituted (location: c) +  (24) cpd00037: UDP_N_acetylglucosamine (location: c) =  (24) cpd00014: UDP (location: c) +

 (1) cpd00061: Phosphoenolpyruvate (location: c) +  (1) cpd00037: UDP_N_acetylglucosamine (location: c) =  (1) cpd02820: UDP_N_acetylglucosamine_enolpyruvate (location: c) +  (1) cpd00067: H (location: c) +  (1) cpd00009: Phosphate (location: c)
 (1) cpd00095: Glycerone_phosphate (location: c) >  (1) cpd00067: H (location: c) +  (1) cpd00009: Phosphate (location: c) +  (1) cpd00428: 2_Oxopropanal (location: c)
 (1) cpd02566: 2_3_Dihydroxy_2_3_dihydrobenzoate (location: c) +  (1) cpd00003: NAD (location: c) >  (1) cpd00168: 2_3_Dihydroxybenzoate (location: c) +  (1) cpd00067: H (location: c) +  (1) cpd00004: NADH (location: c)
 (1) cpd00035: L_Alanine (location: c) =  (1) cpd00117: D_Alanine (location: c)
 (1) cpd00045: Adenosine_3_5_bisphosphate (location: c) +  (1) cpd00001: H2O (location: c) >  (1) cpd00067: H (location: c) +  (1) cpd00018: AMP (location: c) +  (1) cpd00009: Phosphate (location: c)
 (1) cpd00067: H (location: c) +  (1) cpd15414: d_biotin_d_sulfoxide (location: c) +  